## Deliverable 2. Create a Customer Travel Destinations Map.

In [64]:
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import requests
import gmaps
from datetime import datetime
# Import the time module.
import time
# Get today's date in seconds.
today = time.time()
# Import the API key.
from config import weather_api_key

In [65]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,Unnamed: 0,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Picton,AU,2021-10-02 03:12:54,-34.1833,150.6000,72.73,62,84,5.32
1,1,Ushuaia,AR,2021-10-02 03:07:28,-54.8000,-68.3000,35.26,80,40,17.27
2,2,Ponta Do Sol,PT,2021-10-02 03:04:47,32.6667,-17.1000,73.00,79,2,4.94
3,3,Oranjemund,NaN,2021-10-02 03:07:11,-28.5500,16.4333,56.59,89,79,15.64
4,4,Francisco Beltrao,BR,2021-10-02 03:12:56,-26.0811,-53.0550,61.86,97,100,2.17


In [67]:
#2 Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [68]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,Unnamed: 0,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
12,12,Avarua,CK,2021-10-02 03:05:13,-21.2078,-159.7750,78.85,73,100,11.50
16,16,Thinadhoo,MV,2021-10-02 03:12:59,0.5333,72.9333,82.49,69,34,9.01
17,17,Kahului,US,2021-10-02 03:04:25,20.8947,-156.4700,82.42,72,1,25.32
22,22,Tawau,MY,2021-10-02 03:13:02,4.2498,117.8871,83.75,72,96,2.80
28,28,Boyolangu,ID,2021-10-02 03:13:04,-8.1181,111.8935,84.38,56,100,8.23
29,29,San Patricio,US,2021-10-02 03:04:20,28.0170,-97.5169,78.82,80,79,7.11
40,40,Gourcy,BF,2021-10-02 03:13:06,13.2167,-2.3500,78.44,68,94,7.18
42,42,Hilo,US,2021-10-02 03:04:14,19.7297,-155.0900,80.33,76,1,6.91
43,43,Bambanglipuro,ID,2021-10-02 03:05:45,-7.9500,110.2833,82.85,62,99,8.97
49,49,Atuona,PF,2021-10-02 03:04:18,-9.8000,-139.0333,78.31,72,83,21.03


In [69]:
# 4a. Determine if there are any empty rows.
#preferred_cities_df.head()
#preferred_cities_df.isnull().values.any()
preferred_cities_df.isnull().sum().sum()

0

In [70]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = pd.DataFrame(preferred_cities_df)
clean_df.head(10)

,Unnamed: 0,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
12,12,Avarua,CK,2021-10-02 03:05:13,-21.2078,-159.7750,78.85,73,100,11.50
16,16,Thinadhoo,MV,2021-10-02 03:12:59,0.5333,72.9333,82.49,69,34,9.01
17,17,Kahului,US,2021-10-02 03:04:25,20.8947,-156.4700,82.42,72,1,25.32
22,22,Tawau,MY,2021-10-02 03:13:02,4.2498,117.8871,83.75,72,96,2.80
28,28,Boyolangu,ID,2021-10-02 03:13:04,-8.1181,111.8935,84.38,56,100,8.23
29,29,San Patricio,US,2021-10-02 03:04:20,28.0170,-97.5169,78.82,80,79,7.11
40,40,Gourcy,BF,2021-10-02 03:13:06,13.2167,-2.3500,78.44,68,94,7.18
42,42,Hilo,US,2021-10-02 03:04:14,19.7297,-155.0900,80.33,76,1,6.91
43,43,Bambanglipuro,ID,2021-10-02 03:05:45,-7.9500,110.2833,82.85,62,99,8.97
49,49,Atuona,PF,2021-10-02 03:04:18,-9.8000,-139.0333,78.31,72,83,21.03


In [71]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

KeyError: "['Current Description'] not in index"

In [56]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
or index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index,"Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found...  skipping.")
        

SyntaxError: invalid syntax (919230638.py, line 9)

In [57]:
# 7. Drop the rows where there is no Hotel Name.


In [58]:
# 8a. Create the output File (CSV)
hotel_df.to_csv('WeatherPy_vacation.csv')

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

NameError: name 'clean_hotel_df' is not defined

In [59]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """

"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

NameError: name 'clean_hotel_df' is not defined

In [60]:
# 11a. Add a marker layer for each city to the map. 

# 11b. Display the figure
